In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from urllib.parse import quote_plus
import pyodbc

In [2]:
conn = pyodbc.connect(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=DESKTOP-MLG7MPI\MSSQLSERVER01;"
    r"DATABASE=UFCStorage;"
    r"UID=administrator;"
    r"PWD=administrator;"
)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sys.tables")
print(cursor.fetchall())

[('sysdiagrams',), ('dim_Stance',), ('dim_Referee',), ('dim_FinishType',), ('dim_Date',), ('dim_Event',), ('dim_Fighter',), ('fact_Fight',)]


In [3]:
# Create the connection string
username = "administrator"
password = "administrator"
server = "DESKTOP-MLG7MPI\MSSQLSERVER01"
database = "UFCStorage"

# Properly encode the connection string
params = quote_plus(
    f"DRIVER=ODBC Driver 17 for SQL Server;"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [4]:
query = """
SELECT * 
FROM 
(
	SELECT fightID, fighterRedID, fighterBlueID, winner 
	FROM fact_Fight
) AS t
INNER JOIN 
(
	SELECT 
		fighterID, wins as rWins, losses as rLosses, draws as rDraws, 
		height as rHeight, reach as rReach, weight as rWeight,
		stanceID as rStanceID,
		sigStrAcc as rSigStrAcc, sigStrDef as rSigStrDef, 
		tdAcc as rTdAcc, tdDef as rTdDef, avgSubAtt as rAvgSubAtt,
		dateOfBirth as rDOB
	FROM dim_Fighter
) AS r
ON fighterRedID = r.fighterID
INNER JOIN 
(
	SELECT 
		fighterID, wins as bWins, losses as bLosses, draws as bDraws, 
		height as bHeight, reach as bReach, weight as bWeight,
		stanceID as bStanceID,
		sigStrAcc as bSigStrAcc, sigStrDef as bSigStrDef, 
		tdAcc as bTdAcc, tdDef as bTdDef, avgSubAtt as bAvgSubAtt,
		dateOfBirth as bDOB
	FROM dim_Fighter
) AS b
ON fighterBlueID = b.fighterID

SELECT 
	fighterID, wins as rWins, losses as rLosses, draws as rDraws, 
	height as rHeight, reach as rReach, stanceID as rStanceID,
	sigStrAcc as rSigStrAcc, sigStrDef as rSigStrAcc, 
	tdAcc as rTdAcc, tdDef as rTdDef, avgSubAtt as rAvgSubAtt,
	dateOfBirth as rDOB
FROM dim_Fighter"""

df = pd.read_sql(query, engine)
df

,fightID,fighterRedID,fighterBlueID,winner,fighterID,rWins,rLosses,rDraws,rHeight,rReach,...,bHeight,bReach,bWeight,bStanceID,bSigStrAcc,bSigStrDef,bTdAcc,bTdDef,bAvgSubAtt,bDOB
0,1,969,1151,B,969,12,4,0,160,167,...,165,165,56,3,40,62,48,60,0,19920629.0
1,2,2445,3599,R,2445,9,1,0,190,200,...,182,187,119,4,55,49,0,100,0,19931213.0
2,3,1139,1505,R,1139,12,4,0,187,190,...,185,193,83,3,47,47,75,64,0,19920118.0
3,4,1135,932,R,1135,7,0,0,177,177,...,172,170,61,4,46,55,36,44,0,20001220.0
4,5,580,653,B,580,18,5,0,177,177,...,177,182,61,5,48,64,31,60,1,19960904.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7401,7402,2027,3912,R,2027,1,5,0,177,182,...,187,192,111,3,0,0,0,0,0,NaN
7402,7403,2966,3104,R,2966,1,0,0,178,182,...,190,195,95,3,0,0,0,0,0,NaN
7403,7404,3223,2212,R,3223,2,1,0,182,187,...,195,200,124,3,0,0,0,0,0,NaN
7404,7405,3506,3565,R,3506,20,17,0,187,192,...,178,182,77,3,0,0,0,0,0,NaN


In [25]:
df['winner'] = df['winner'].replace({'R': 1, 'B': 0})

C:\Users\Svetlana Nazarko\AppData\Local\Temp\ipykernel_18184\923928451.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['winner'] = df['winner'].replace({'R': 1, 'B': 0})


In [26]:
threshold = 0.6

# Compute correlation matrix
corr_matrix = df.corr().abs()

# Mask the upper triangle and diagonal (to avoid duplicate pairs)
mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
filtered_corr = corr_matrix.where(mask)

# Find pairs with correlation above threshold
high_corr_pairs = (
    filtered_corr.stack()
    .reset_index()
    .rename(columns={'level_0': 'Feature 1', 'level_1': 'Feature 2', 0: 'Correlation'})
    .query('Correlation > @threshold')
    .sort_values(by='Correlation', ascending=False)
)

print(high_corr_pairs)

         Feature 1  Feature 2  Correlation
33    fighterRedID  fighterID     1.000000
76   fighterBlueID  fighterID     1.000000
278        rWeight    bWeight     0.911139
451        bHeight     bReach     0.902922
220        rHeight     rReach     0.899711
221        rHeight    rWeight     0.794070
452        bHeight    bWeight     0.772316
243         rReach    rWeight     0.763813
276        rWeight    bHeight     0.754381
30         fightID       bDOB     0.750220
235        rHeight    bWeight     0.747321
460         bReach    bWeight     0.745572
233        rHeight    bHeight     0.727793
277        rWeight     bReach     0.727506
16         fightID       rDOB     0.723940
257         rReach    bWeight     0.717749
234        rHeight     bReach     0.707214
255         rReach    bHeight     0.704480
256         rReach     bReach     0.688071
418          bWins    bLosses     0.675999
145          rWins    rLosses     0.654433
404           rDOB       bDOB     0.650219


In [27]:
columns_to_drop = ['fightID', 'fighterRedID', 'fighterBlueID', 'fighterID', 'winner']
X = df.drop(columns=columns_to_drop)
y = df['winner']

In [28]:
X = X.fillna(X.mean(numeric_only=True))

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

C:\Users\Svetlana Nazarko\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [38]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.689608636977058

Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.23      0.32       477
           1       0.71      0.91      0.80      1005

    accuracy                           0.69      1482
   macro avg       0.63      0.57      0.56      1482
weighted avg       0.66      0.69      0.65      1482



In [39]:
r_counter = 0
b_counter = 0
for i in y_pred:
    if i == 0:
        b_counter += 1
    else:
        r_counter += 1
print('R:', r_counter)
print('B:', b_counter)

R: 1279
B: 203


In [40]:
df['rDOB'] = pd.to_datetime(df['rDOB'].dropna().astype(int).astype(str), format='%Y%m%d', errors='coerce')
df['bDOB'] = pd.to_datetime(df['bDOB'].dropna().astype(int).astype(str), format='%Y%m%d', errors='coerce')

reference_date = pd.to_datetime('2030-06-30')
df['rAge'] = (reference_date - df['rDOB']).dt.days 
df['bAge'] = (reference_date - df['bDOB']).dt.days 

In [41]:
df['reach_diff'] = df['rReach'] - df['bReach']
df['height_diff'] = df['rHeight'] - df['bHeight']
df['weight_diff'] = df['rWeight'] - df['bWeight']

df['sig_str_acc_diff'] = df['rSigStrAcc'] - df['bSigStrAcc']
df['sig_str_def_diff'] = df['rSigStrDef'] - df['bSigStrDef']
df['td_acc_diff'] = df['rTdAcc'] - df['bTdAcc']
df['td_def_diff'] = df['rTdDef'] - df['bTdDef']

df['sub_att_diff'] = df['rAvgSubAtt'] - df['bAvgSubAtt']
df['wins_diff'] = df['rWins'] - df['bWins']
df['losses_diff'] = df['rLosses'] - df['bLosses']
df['experience_diff'] = (df['rWins'] + df['rLosses'] + df['rDraws']) - (df['bWins'] + df['bLosses'] + df['bDraws'])
df['win_ratio_diff'] = (df['rWins'] / (df['rWins'] + df['rLosses'] + 1)) - (df['bWins'] / (df['bWins'] + df['bLosses'] + 1))

df['age_days_diff'] = df['rAge'] - df['bAge']
df['age_days_diff'] = df['age_days_diff'].fillna(df['age_days_diff'].median())

In [42]:
columns_to_drop = [col for col in df.columns if col.startswith('r') or col.startswith('b')]
X = df.drop(columns=columns_to_drop + ['fightID', 'fighterRedID', 'fighterBlueID', 'fighterID', 'winner'])

In [43]:
X

,height_diff,weight_diff,sig_str_acc_diff,sig_str_def_diff,td_acc_diff,td_def_diff,sub_att_diff,wins_diff,losses_diff,experience_diff,win_ratio_diff,age_days_diff
0,-5,-4,2,1,3,27,0,0,-2,-2,0.074303,-423.0
1,8,-13,-6,13,46,0,0,2,-2,0,0.181818,1437.0
2,2,0,4,-4,-36,-1,0,5,2,7,0.005882,-2133.0
3,5,0,5,-4,-36,44,0,-2,-1,-3,0.056818,833.0
4,0,4,9,-21,-8,3,0,8,0,7,0.125000,2827.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7401,-10,-34,0,0,0,0,0,1,4,5,0.142857,124.0
7402,-12,-18,0,0,0,0,0,1,-1,0,0.500000,124.0
7403,-13,-29,0,0,0,0,0,1,-2,-1,0.300000,124.0
7404,9,25,0,0,0,0,0,20,16,36,0.526316,124.0


In [44]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\n", classification_report(y_test, y_pred))

Accuracy: 0.7294197031039136

               precision    recall  f1-score   support

           0       0.60      0.48      0.54       477
           1       0.78      0.85      0.81      1005

    accuracy                           0.73      1482
   macro avg       0.69      0.67      0.67      1482
weighted avg       0.72      0.73      0.72      1482



In [45]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.680161943319838
              precision    recall  f1-score   support

           0       0.50      0.69      0.58       477
           1       0.82      0.68      0.74      1005

    accuracy                           0.68      1482
   macro avg       0.66      0.68      0.66      1482
weighted avg       0.72      0.68      0.69      1482



C:\Users\Svetlana Nazarko\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
probs = model.predict_proba(X_test)
pd.DataFrame([(probs[i, 0], probs[i, 1], y_test.iat[i]) for i in range(len(probs))]).head(60)

,0,1,2
0,0.512253,0.487747,1
1,0.315347,0.684653,1
2,0.508782,0.491218,1
3,0.143495,0.856505,1
4,0.617012,0.382988,0
5,0.237572,0.762428,1
6,0.519417,0.480583,1
7,0.475440,0.524560,1
8,0.441407,0.558593,1
9,0.608148,0.391852,0


In [22]:
X.corr()

,height_diff,weight_diff,sig_str_acc_diff,sig_str_def_diff,td_acc_diff,td_def_diff,sub_att_diff,wins_diff,losses_diff,experience_diff,win_ratio_diff,age_days_diff
height_diff,1.000000,0.207559,0.062805,-0.054911,0.036200,-0.017184,0.108693,0.004973,-0.011764,-0.001216,0.037137,-0.131506
weight_diff,0.207559,1.000000,-0.015226,-0.082201,0.014302,0.002991,0.003485,0.016047,-0.013093,0.003494,0.027859,-0.026153
sig_str_acc_diff,0.062805,-0.015226,1.000000,0.284044,0.262984,0.171683,0.078284,0.065711,-0.043695,0.033616,0.239879,-0.048848
sig_str_def_diff,-0.054911,-0.082201,0.284044,1.000000,0.208152,0.326289,-0.118631,0.082474,0.007078,0.064477,0.189678,0.019701
td_acc_diff,0.036200,0.014302,0.262984,0.208152,1.000000,0.263955,0.007777,0.069972,0.009346,0.054389,0.127367,-0.057492
td_def_diff,-0.017184,0.002991,0.171683,0.326289,0.263955,1.000000,-0.212548,0.045200,0.001956,0.033830,0.132426,-0.089048
sub_att_diff,0.108693,0.003485,0.078284,-0.118631,0.007777,-0.212548,1.000000,0.003411,-0.051649,-0.016797,0.070626,-0.026237
wins_diff,0.004973,0.016047,0.065711,0.082474,0.069972,0.045200,0.003411,1.000000,0.605083,0.956136,0.302849,0.086465
losses_diff,-0.011764,-0.013093,-0.043695,0.007078,0.009346,0.001956,-0.051649,0.605083,1.000000,0.807687,-0.399137,0.246735
experience_diff,-0.001216,0.003494,0.033616,0.064477,0.054389,0.033830,-0.016797,0.956136,0.807687,1.000000,0.080178,0.154933


In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

knn_params = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2, 3, 4]
}

knn = KNeighborsClassifier()
knn_grid = GridSearchCV(knn, knn_params, scoring='accuracy')
knn_grid.fit(X_train, y_train)
print("Best KNN params:", knn_grid.best_params_)

Best KNN params: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}


In [47]:
model = knn_grid.best_estimator_

y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.655195681511471
              precision    recall  f1-score   support

           0       0.45      0.35      0.39       477
           1       0.72      0.80      0.76      1005

    accuracy                           0.66      1482
   macro avg       0.59      0.57      0.58      1482
weighted avg       0.64      0.66      0.64      1482



In [48]:
from sklearn.ensemble import RandomForestClassifier

rf_params = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(rf, rf_params, cv=5, scoring='accuracy')
rf_grid.fit(X_train, y_train)
print("Best Random Forest params:", rf_grid.best_params_)

Best Random Forest params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


In [49]:
model = rf_grid.best_estimator_

y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7300944669365722
              precision    recall  f1-score   support

           0       0.61      0.46      0.52       477
           1       0.77      0.86      0.81      1005

    accuracy                           0.73      1482
   macro avg       0.69      0.66      0.67      1482
weighted avg       0.72      0.73      0.72      1482



In [50]:
from sklearn.linear_model import LogisticRegression

logreg_params = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'max_iter': [100, 200, 1000]
}

logreg = LogisticRegression(random_state=42)
logreg_grid = GridSearchCV(logreg, logreg_params, cv=5, scoring='accuracy')
logreg_grid.fit(X_train, y_train)
print("Best Logistic Regression params:", logreg_grid.best_params_)

C:\Users\Svetlana Nazarko\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Svetlana Nazarko\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https

Best Logistic Regression params: {'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}


In [51]:
model = logreg_grid.best_estimator_

y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7314439946018894
              precision    recall  f1-score   support

           0       0.62      0.44      0.51       477
           1       0.77      0.87      0.81      1005

    accuracy                           0.73      1482
   macro avg       0.69      0.65      0.66      1482
weighted avg       0.72      0.73      0.72      1482

